# Building Multi-Agent Investment Firms with Strands SDK

## Executive Summary

This notebook demonstrates the implementation of a sophisticated multi-agent system for financial market analysis using the Strands Agents SDK. The system replicates the collaborative dynamics of real-world trading firms, incorporating specialized agents for fundamental analysis, sentiment evaluation, technical analysis, trading execution, and risk management.

### Learning Objectives
- Master the fundamentals of the Strands Agent API
- Implement market data retrieval and analysis workflows
- Create specialized agents for different analytical functions
- Generate actionable investment reports and recommendations

### System Architecture
- **Multi-Agent Framework**: Coordinated agents with specialized expertise
- **Real-time Data Integration**: Live market feeds and historical analysis
- **Technical Analysis Engine**: Comprehensive indicator-based evaluation
- **Automated Reporting**: Structured output for decision-making

---

## Task 1: Market Analysis Agent Implementation

### Objective
Develop proficiency with the Strands Agent API by creating specialized agents for comprehensive market analysis.

### Requirements
- **Target Stock**: AMZN (Amazon.com Inc.)
- **Analysis Date**: 2025-08-19
- **Agent Types**: Market Report Agent, News Analysis Agent
- **Data Sources**: Historical stock data, market indicators, news sentiment
- **Output Format**: CSV files for quantitative data, text files for qualitative analysis

### Success Criteria
✅ Successful data retrieval from multiple sources  
✅ Comprehensive technical analysis with actionable insights  
✅ Professional report generation with structured recommendations  
✅ CSV and text file outputs for further analysis  

---

## 1. Environment Setup and Dependencies

### Installation Overview
This section installs all required dependencies for the multi-agent financial analysis system. The installation process may show dependency resolution warnings, which can typically be ignored as they don't affect functionality.

### Key Dependencies
- **strands**: Core agent framework for building intelligent agents
- **boto3**: AWS SDK for accessing Bedrock AI models
- **pandas**: Data manipulation and analysis library
- **yfinance**: Yahoo Finance API for market data retrieval
- **stockstats**: Technical indicator calculation library
- **python-dotenv**: Environment variable management

**Note**: If you encounter "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.", you can ignore it as it doesn't affect functionality.

In [ ]:
# Install all required dependencies from requirements.txt
# Note: Dependency resolver warnings can be safely ignored
%pip install -r requirements.txt

## 2. System Configuration and Initialization

### 2.1 Core Library Imports

Import essential libraries for system operation, including configuration management, logging, and environment handling. These imports establish the foundation for our multi-agent system.

In [ ]:
# Core system imports for multi-agent framework
import sys
import os
from pathlib import Path
from default_config import DEFAULT_CONFIG
import logging
from dotenv import load_dotenv

# Configure logging for system monitoring and debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Load environment variables from .env file if present
load_dotenv()

print("✅ Core libraries imported successfully")
print("✅ Logging configured for system monitoring")
print("✅ Environment variables loaded")

### 2.2 AI Model Configuration

Define the AI models that will power our intelligent agents. We use Amazon's latest models for optimal performance:

- **Nova Pro**: Advanced reasoning model for complex financial analysis
- **Titan Embed**: High-quality text embeddings for semantic understanding

In [ ]:
# AI Model Configuration Constants
NOVA_PRO_MODEL_ID = "amazon.nova-pro-v1:0"        # Primary reasoning model

# Model Selection Rationale:
# - Nova Pro: State-of-the-art reasoning for complex financial analysis

print(f"🤖 Primary Model: {NOVA_PRO_MODEL_ID}")
print("✅ AI model configuration completed")

### 2.3 Bedrock Model Factory

This factory function creates optimally configured Bedrock model instances for financial analysis tasks. The configuration includes:

- **Extended Timeouts**: Accommodates complex analytical processes
- **Adaptive Retry Logic**: Ensures reliability in production environments
- **Thinking Mode**: Provides transparency in AI reasoning (optional)
- **Optimized Parameters**: Balanced for financial analysis accuracy

In [ ]:
import boto3
from botocore.config import Config
from strands.models import BedrockModel

# AWS Boto3 client configuration optimized for financial analysis
boto_client_config = Config(
    read_timeout=1800,      # 30 minutes - accommodates complex analyses
    connect_timeout=900,    # 15 minutes - ensures reliable connections
    retries=dict(max_attempts=3, mode="adaptive"),  # Adaptive retry strategy
)

def get_model(model_id=NOVA_PRO_MODEL_ID, thinking=False,
              temperature=0.7, max_tokens=10000):
    """
    Create and return a Bedrock model instance optimized for financial analysis.

    Parameters:
    -----------
    model_id : str
        The specific AWS Bedrock model identifier to use
    thinking : bool
        Enable thinking mode for supported models (provides reasoning transparency)
    temperature : float
        Sampling temperature (0.0-1.0) - controls randomness in responses
        - 0.7 provides good balance between creativity and consistency
    max_tokens : int
        Maximum tokens in the response (affects response length)

    Returns:
    --------
    BedrockModel
        Configured model instance ready for agent integration
        
    Notes:
    ------
    - Temperature 0.7 balances analytical precision with adaptive reasoning
    - Thinking mode provides insight into the model's reasoning process
    - Extended token limit accommodates comprehensive financial reports
    """
    session = boto3.Session()

    # Configure thinking mode for enhanced reasoning transparency
    additional_request_fields = {}
    if thinking:
        additional_request_fields = {
            "thinking": {
                "type": "enabled",
                "budget_tokens": 4096,  # Dedicated tokens for reasoning process
            }
        }

    return BedrockModel(
        model_id=model_id,
        boto_session=session,
        max_tokens=max_tokens,
        temperature=temperature,
        boto_client_config=boto_client_config,
        additional_request_fields=additional_request_fields,
    )

print("✅ Bedrock model factory configured")
print("🔧 Optimized for financial analysis workloads")
print("⚙️ Extended timeouts and adaptive retry logic enabled")

### 2.4 Primary Model Initialization

Initialize the main language model that will serve as the cognitive engine for our financial analysis agents.

In [ ]:
# Initialize the primary language model for agent operations
llm = get_model(model_id=NOVA_PRO_MODEL_ID)

# Verify successful initialization
print(f"🧠 Language Model Initialized: {NOVA_PRO_MODEL_ID}")
print(f"🎛️ Configuration: Temperature=0.7, Max Tokens=10,000")
print(f"✅ Model ready for agent integration")

## 3. Market Analysis Agent Architecture

### 3.1 Agent Design Philosophy

Our Market Analyst agent is architected with institutional-grade capabilities:

- **Multi-Source Data Integration**: Yahoo Finance, technical indicators, market sentiment
- **Adaptive Analysis Framework**: Context-aware indicator selection
- **Risk-Aware Decision Making**: Volatility assessment and position sizing
- **Professional Reporting**: Investment-grade analysis and recommendations

### 3.2 Technical Indicator Arsenal

The agent has access to a comprehensive suite of technical indicators:

**Trend Analysis**: Moving averages (SMA, EMA) for trend identification  
**Momentum Indicators**: MACD system for momentum analysis  
**Oscillators**: RSI for overbought/oversold conditions  
**Volatility Measures**: Bollinger Bands and ATR for risk assessment  
**Volume Analysis**: VWMA for trend confirmation  

### 3.3 Market Analyst Implementation

The following function creates a Market Analyst agent that combines a language model with financial data tools to perform technical analysis and market trend evaluation. Depending on the online flag, it either uses real-time market data and indicators (via Yahoo Finance and technical reports) or switches to offline/cached data sources.

In [ ]:
from strands import Agent
from tools import (
    get_yfin_data_online,
    get_stockstats_indicators_report_online,
    get_yfin_data,
    get_stockstats_indicators_report,
)

def create_market_analyst(llm, online=False):
    """
    Create a Market Analyst agent for technical analysis and market trend evaluation.
    
    Parameters:
    -----------
    llm : BedrockModel
        The language model instance for agent reasoning
    online : bool
        Data source mode:
        - True: Real-time market data and live indicators
        - False: Cached/historical data for backtesting
        
    Returns:
    --------
    Agent
        Configured Market Analyst agent with specialized financial expertise
    """
    # Select tools based on online/offline mode
    if online:
        tools = [
            get_yfin_data_online,                    # Real-time Yahoo Finance data
            get_stockstats_indicators_report_online, # Real-time technical indicators
        ]
    else:
        tools = [
            get_yfin_data,                    # Cached Yahoo Finance data
            get_stockstats_indicators_report, # Cached technical indicators
        ]

    # Define the agent's system prompt with detailed instructions
    system_message = (
        """You are a Market Analyst specializing in technical analysis and market trend evaluation. 
        Your role is to analyze financial markets by selecting the **most relevant technical indicators** 
        for a given market condition or trading strategy.

        **Key Responsibilities:**
        1. Select up to **8 complementary indicators** that provide diverse insights without redundancy
        2. Analyze market trends, momentum, volatility, and volume patterns
        3. Provide actionable insights for trading decisions
        4. Identify support/resistance levels and trend directions

        **Available Technical Indicators by Category:**

        **Moving Averages:**
        - close_50_sma: 50-period Simple Moving Average for medium-term trend analysis
        - close_200_sma: 200-period SMA for long-term trend confirmation and golden/death cross setups
        - close_10_ema: 10-period Exponential Moving Average for short-term momentum shifts

        **MACD (Moving Average Convergence Divergence):**
        - macd: Main MACD line showing momentum via EMA differences
        - macds: MACD Signal line (smoothed EMA) for crossover signals
        - macdh: MACD Histogram showing momentum strength and early divergence signals

        **Momentum Indicators:**
        - rsi: Relative Strength Index for overbought/oversold conditions (70/30 thresholds)

        **Volatility Indicators:**
        - boll: Bollinger Bands Middle (20-period SMA baseline)
        - boll_ub: Bollinger Upper Band (overbought zones and breakout signals)
        - boll_lb: Bollinger Lower Band (oversold conditions)
        - atr: Average True Range for volatility measurement and risk management

        **Volume-Based Indicators:**
        - vwma: Volume Weighted Moving Average for trend confirmation with volume analysis

        **Analysis Guidelines:**
        1. Always call get_yfin_data first to retrieve the necessary price data
        2. Select indicators that provide complementary information (avoid redundancy)
        3. Explain why each selected indicator is suitable for the current market context
        4. Provide detailed, nuanced analysis rather than generic "mixed trends" statements
        5. Focus on actionable insights that help traders make informed decisions
        6. Include a markdown table at the end summarizing key findings

        **Output Format:**
        - Detailed technical analysis with specific observations
        - Clear trend identification and momentum assessment
        - Support/resistance level identification
        - Risk assessment and volatility analysis
        - Summary table with key metrics and recommendations
        """
    )

    # Create and configure the agent
    agent = Agent(
        model=llm,
        tools=tools,
        name="Market Analyst",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )

    return agent

### 3.4 Agent Operational Configuration

Configure the Market Analyst agent for real-time analysis mode. This ensures access to the most current market data and live technical indicators for accurate, up-to-date analysis.

Here we will use online analysis to access real-time market data and indicators.

In [ ]:
# Configure agent for real-time market analysis
online = True  # Enable real-time data feeds
market_agent = create_market_analyst(llm, online)

# Verify agent configuration
print(f"🤖 Agent Initialized: {market_agent.name}")
print(f"📡 Data Mode: {'Real-time' if online else 'Historical'} market data")
#print(f"🛠️ Available Tools: {len(market_agent.tools)} specialized financial analysis tools")
print(f"✅ Market Analyst ready for comprehensive analysis")

## 4. Analysis Configuration and Parameters

### 4.1 Target Stock Code Selection

Define the specific stock code and analysis parameters for our comprehensive market study. Amazon (AMZN) represents an ideal case study as a large-cap technology stock with high liquidity and significant market influence.

Set the target stock ticker `AMZN` and the specific trade date `2025-08-19` for analysis.

In [ ]:
# Analysis Target Configuration
company_of_interest = "AMZN"  # Stock ticker to analyze
trade_date = "2025-08-19"    # Analysis date

# Display configuration summary
print("📊 ANALYSIS TARGET CONFIGURATION")
print("=" * 40)
print(f"Target Stock Code   : {company_of_interest}")
print(f"Company Name      : Amazon.com Inc.")
print(f"Exchange          : NASDAQ")
print(f"Sector            : Technology")
print(f"Analysis Date     : {trade_date}")
print("=" * 40)
print("✅ Target Stock configured for analysis")

### 4.2 Output Directory Structure

Establish a professional directory structure for organizing analysis outputs. This systematic approach ensures organized storage and easy retrieval of generated reports and data files.

Prepare `working_dir`, default location is the directory `results`.

In [ ]:
# Directory structure configuration
working_dir = DEFAULT_CONFIG['results_dir']  # Base results directory
prefix = f"{company_of_interest}_{trade_date}".replace(" ", "_")  # Unique identifier
config = DEFAULT_CONFIG.copy()  # Configuration backup

# Display directory structure
print("📁 OUTPUT DIRECTORY STRUCTURE")
print("=" * 40)
print(f"Base Directory    : {working_dir}")
print(f"Analysis Prefix   : {prefix}")
print("=" * 40)
print("✅ Professional directory structure configured")

## 5. Market Analysis Execution

### 5.1 Comprehensive Technical Analysis

Execute our sophisticated market analysis using the specialized Market Analyst agent. This process encompasses:

1. **Real-time Data Retrieval**: Current market data and historical context
2. **Multi-Indicator Analysis**: Strategic selection of 8 complementary indicators
3. **Trend Assessment**: Multi-timeframe trend strength evaluation
4. **Risk Quantification**: Volatility analysis and position sizing guidance
5. **Professional Reporting**: Investment-grade analysis with actionable insights

### Starting Analysis

In [ ]:
print(f"🚀 Starting analysis for {company_of_interest} on {trade_date}")
print("=" * 50)
print(f"Target Stock Code   : {company_of_interest}")
print(f"Analysis Date     : {trade_date}")
print(f"Data Mode         : {'Real-time' if online else 'Historical'}")
print(f"Agent             : {market_agent.name}")
print("=" * 50)
print("\n🔄 Processing market data and technical indicators...\n")

# Construct comprehensive analysis prompt
prompt = f"Analyze the market for {company_of_interest} for the trade date {trade_date}"

# Execute comprehensive market analysis
result = market_agent(prompt)

print(f"\n✅ ANALYSIS COMPLETED SUCCESSFULLY")
print(f"📊 Report Generated: {len(str(result)):,} characters")
print(f"🎯 Professional-grade analysis ready for review")

### 5.2 Professional Report Management

Implement a robust file management system for storing analysis outputs with comprehensive metadata and audit trails.

Save analysis result to structured file system for future reference and analysis.

In [ ]:
def save_as_file(text, working_dir, prefix='', file_name=''):
    """
    Save text content to a file in the specified directory structure.
    
    Parameters:
    -----------
    text : str
        Content to save
    working_dir : str
        Base working directory
    prefix : str
        Subdirectory prefix (usually ticker_date)
    file_name : str
        Name of the file to save
        
    Features:
    ---------
    - Automatic directory creation
    - UTF-8 encoding for international character support
    - Error handling and validation
    """
    # Create directory if it doesn't exist
    full_dir = os.path.join(working_dir, prefix)
    if not os.path.exists(full_dir):
        os.makedirs(full_dir, exist_ok=True)
    
    # Write content to file
    file_path = os.path.join(full_dir, file_name)
    with open(file_path, "w", encoding='utf-8') as f:
        f.write(text)
    
    print(f"✅ Report saved successfully: {file_path}")
    return file_path

In [ ]:
# Save the comprehensive market analysis report
save_as_file(str(result), working_dir, prefix, "market_report.txt")

print("📊 MARKET ANALYSIS REPORT SAVED SUCCESSFULLY")
print("=" * 50)
print(f"File Size         : {len(str(result)):,} characters")
print(f"Stock Code          : {company_of_interest}")
print(f"Analysis Date     : {trade_date}")
print(f"Data Mode         : {'Real-time' if online else 'Historical'}")
print("=" * 50)
print("✅ Professional report with audit trail completed")

## 6. News Analysis

The following function creates a News Analyst agent that uses a language model and selected news sources (real-time or cached) to analyze financial news, identify macroeconomic and geopolitical trends, assess market sentiment, and generate actionable insights for trading decisions based on a structured analysis framework.

<p style="color:red; font-weight:bold; font-size:16pt"> ⚠️ You need to put the agent tool in the correct position to run the codes successfully!</p>

In [ ]:
from tools import get_google_news


def create_news_analyst(llm):
    """
    Create a News Analyst agent for news analysis and macroeconomic trend evaluation.
    
    Args:
        llm: The language model to use for the agent
        
    Returns:
        Agent: Configured News Analyst agent
    """
    

    # Define the agent's system prompt with comprehensive instructions
    system_message = (
        """You are a News Analyst specializing in financial news analysis and macroeconomic research. 
        Your role is to analyze recent news and global trends to provide context for trading decisions.

        **Key Responsibilities:**
        1. Analyze recent news and events from the past week that impact financial markets
        2. Identify macroeconomic trends and their potential market implications
        3. Assess geopolitical events and their effects on different sectors
        4. Evaluate market sentiment from news coverage and social media
        5. Provide actionable insights for trading strategies

        **Analysis Framework:**
        
        **Market-Moving Events:**
        - Central bank announcements and monetary policy changes
        - Economic data releases (GDP, inflation, employment, etc.)
        - Corporate earnings and guidance updates
        - Geopolitical developments and trade relations
        - Regulatory changes and policy announcements

        **Sector-Specific Impact:**
        - Technology: Innovation, regulation, competitive dynamics
        - Healthcare: Drug approvals, policy changes, demographic trends
        - Energy: Commodity prices, environmental policies, supply chain
        - Financial: Interest rates, regulatory changes, credit conditions
        - Consumer: Spending patterns, inflation impact, demographic shifts

        **Sentiment Analysis:**
        - Overall market sentiment (bullish, bearish, neutral)
        - Investor confidence indicators
        - Risk appetite and flight-to-quality movements
        - Social media sentiment and retail investor behavior

        **Global Context:**
        - International market developments
        - Currency movements and their implications
        - Cross-border trade and supply chain impacts
        - Regional economic conditions and policies

        **Analysis Guidelines:**
        1. Focus on news with clear market implications
        2. Distinguish between short-term noise and long-term trends
        3. Provide specific, actionable insights rather than generic observations
        4. Consider both direct and indirect effects on different asset classes
        5. Assess the reliability and credibility of news sources
        6. Include a markdown table summarizing key findings and implications

        **Output Requirements:**
        - Comprehensive analysis of market-relevant news and trends
        - Clear identification of potential trading opportunities and risks
        - Sector-specific impact assessment
        - Timeline of key events and their market implications
        - Summary table with key news items, impact assessment, and trading implications
        """
    )

    # Create and configure the agent
    # Fill in the tool parameters below to enable real-time news analysis
    # Refer to the imported tools above in this code block to select the appropriate tool
    agent = Agent(
        model=llm,
        # ⚠️ IMPORTANT: Read this code block and repalce XXX_tool with the correct tool.
        # Example: tools=[search_tool]
        tools=[XXX_tool],
        name="News Analyst",
        system_prompt=system_message,
        load_tools_from_directory=False,
    )

    return agent


### 6.1 Create news agent

In [ ]:
news_analyst_agent = create_news_analyst(llm)

### 6.2 Generate news report

In [ ]:
print("Running news analysis...")
prompt = f"Analyze the news for {company_of_interest} for the trade date {trade_date}"
result = news_analyst_agent(prompt)
save_as_file(str(result), working_dir,prefix, "news_report.txt")
print("News analysis completed.")

### 6.3 Get used tool

In [ ]:
used_tools = []
if hasattr(result, 'metrics') and hasattr(result.metrics, 'tool_metrics'):
    for tool_name, tool_metric in result.metrics.tool_metrics.items():
        if tool_metric.call_count > 0:
            used_tools.append(tool_name)
print(used_tools[0])

## 7. Stock Data Retrieval and Validation

Retrieve and validate the stock data that is generated during our analysis process. This data serves as the foundation for all technical indicators and analysis conclusions.

### 7.1 Locate stock data directory
The stock data is automatically cached in the `./results/data_cache/` directory during the analysis process.

In [ ]:
# Define data cache directory for retrieved market data
stock_data_dir = './results/data_cache/'

print(f"📂 Data Cache Directory: {stock_data_dir}")
print(f"🔍 Searching for generated data files...")

# List available data files
if os.path.exists(stock_data_dir):
    data_files = [f for f in os.listdir(stock_data_dir) if f.endswith('.csv')]
    print(f"📄 Available data files: {len(data_files)}")
    for file in data_files[:5]:  # Show first 5 files
        print(f"   • {file}")
    if len(data_files) > 5:
        print(f"   ... and {len(data_files) - 5} more files")
else:
    print(f"⚠️ Data cache directory not found. Creating directory...")
    os.makedirs(stock_data_dir, exist_ok=True)

### 7.2 Interactive Data File Selection

Provide an interactive interface for selecting the specific data file generated during our analysis. This ensures we're working with the correct dataset.

**Expected filename pattern**: `TICKER-YFin-data-YYYY-MM-DD-YYYY-MM-DD.csv`  
**Example**: `AMZN-YFin-data-2010-09-07-2025-09-07.csv`

In [ ]:
# Interactive file selection for data validation
fn = input("Enter the file name")
    
print(f"✅ Selected file: {fn}")

### 7.3 Display Selected Filename

Confirm the filename that was entered for data loading and validation.

In [ ]:
# Display the selected filename for confirmation
print(f"📄 Selected Data File: {fn}")
print(f"📂 Full Path: {stock_data_dir + fn}")
fn

### 7.4 Data Loading and Display

Load the selected dataset and display all data to verify the market data structure and content.

In [ ]:
import pandas as pd
try:
    # Load market data
    df = pd.read_csv(stock_data_dir + fn)
    
    print(f"📊 DATA LOADING SUMMARY")
    print("=" * 40)
    print(f"File: {fn}")
    print(f"Records: {len(df):,} trading days")
    print(f"Columns: {list(df.columns)}")
    print(f"Date Range: {df['Date'].min()} to {df['Date'].max()}")
    print("=" * 40)
    print("✅ Data loaded successfully")
    
except FileNotFoundError:
    print(f"❌ File not found: {fn}")
    df = None
except Exception as e:
    print(f"❌ Error loading data: {e}")
    df = None

In [ ]:
# Display all data from the CSV file
if df is not None:
    print(f"\n📋 COMPLETE DATASET ({df.shape[0]:,} rows × {df.shape[1]} columns)")
    display(df)
else:
    print("⚠️ No data available for display")

### 7.5 Target Date Analysis

Filter the dataset to show only the specified date stock information for detailed analysis.

In [ ]:
pd.options.display.float_format = '{:.14f}'.format

filtered_df = df[(df['Date'] == '2025-08-18') ]
filtered_df

## 8. Store these variables to reuse in subsequent challenges

In [ ]:
%store NOVA_PRO_MODEL_ID
%store company_of_interest
%store trade_date
%store working_dir
%store prefix
%store config

## 9. You can now navigate to the JAM Task 1 page and complete the challenge
